In [1]:
%pwd 

'/workspaces/tensorflow-wavelets/Development/OpenDVC'

In [2]:
# %cd /home/ubu-admin/Developer/tensorflow-wavelets
%cd /workspaces/tensorflow-wavelets
import OpenDVC
import OpenDVC2
import numpy as np
import load
import tensorflow as tf
import matplotlib.pyplot as plt
import DataGen
import Callbacks
import datetime

/workspaces/tensorflow-wavelets


2022-02-06 18:12:55.540292: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
batch_size = 1
EPOCHS = 10000
STEPS_PER_EPOCH = 60
Height = 240
Width = 240
Channel = 3
lmbda = 1024
lr_init = 1e-7
# samples = 1200
early_stop = 100
I_QP=27

args = OpenDVC.Arguments()
last = 9
checkponts_last_path = "checkpoints_Lagrangian_256_{}/".format(last)
checkponts_new_path = "checkpoints_Lagrangian_256_{}/".format(last+1)

save_name = "model_save_l_256_" + str(last+1)


In [4]:
%load_ext tensorboard


In [5]:
rm -rf ./logs/

In [6]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [7]:
model = OpenDVC.OpenDVC(width=Width, height=Height, batch_size=batch_size, num_filters=128, lmbda=lmbda)
# model = OpenDVC2.OpenDVC(width=Width, height=Height, batch_size=batch_size, num_filters=128, lmbda=lmbda)
# model = OpenDVC2.OpenDVC_SSIM(width=Width, height=Height, batch_size=batch_size, num_filters=128, lmbda=lmbda)
# model = OpenDVC2.OpenDVC_NORM(width=Width, height=Height, batch_size=batch_size, num_filters=128, lmbda=lmbda)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_init),)
print("* [Model compiled]...")

print("* [Loading dataset]...")
data = DataGen.DataVimeo90kGenerator("/mnt/WindowsDev/Developer/tensorflow-wavelets/folder_cloud.npy", 
                                    batch_size,
                                    (Height,Width,Channel),
                                    Channel,
                                    True, 
                                    I_QP,
                                    True)

print("Loading weights")



model.load_weights(checkponts_last_path)
print("Going to fit")

2022-02-06 18:12:59.071080: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-02-06 18:12:59.106939: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-06 18:12:59.107711: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:09.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.852GHz coreCount: 28 deviceMemorySize: 11.77GiB deviceMemoryBandwidth: 335.32GiB/s
2022-02-06 18:12:59.107754: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-02-06 18:12:59.114856: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-02-06 18:12:59.114925: I tensorflow/stream_executor/pl

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


2022-02-06 18:13:02.801902: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.10


* [Model compiled]...
* [Loading dataset]...
Loading weights


2022-02-06 18:13:08.159973: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Going to fit


In [8]:
# model.layers[0].trainable = False
# model.layers[1].trainable = False
# model.layers[2].trainable = True
# model.layers[3].trainable = True
# model.layers[4].trainable = False
# model.layers[5].trainable = True

In [9]:
for layer in model.layers:
    print(layer.name, layer.trainable)

mv_analysis True
mv_synthesis True
res_analysis True
res_synthesis True
optical_flow True
motion_compensation True


In [10]:
# tf.config.run_functions_eagerly(True)
hist = model.fit(x=data, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, verbose=1, batch_size=batch_size,
                callbacks=[
                    Callbacks.MemoryCallback(),
                    # Callbacks.LearningRateReducer(),
                    tf.keras.callbacks.ModelCheckpoint(filepath=checkponts_new_path, save_weights_only=True, save_freq='epoch', monitor="loss", mode='min',  save_best_only=True, verbose=1), 
                    tf.keras.callbacks.TerminateOnNaN(),
                    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=early_stop),
                    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, update_freq="epoch"),            
                    ],
				)

2022-02-06 18:13:08.945436: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-02-06 18:13:08.945495: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-02-06 18:13:08.945587: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2022-02-06 18:13:08.947759: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcupti.so.11.2
2022-02-06 18:13:09.282731: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-02-06 18:13:09.286216: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1743] CUPTI activity buffer flushed
2022-02-06 18:13:09.542939: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-06 18:13:09.543793: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2499995000 Hz


Epoch 1/10000


2022-02-06 18:13:33.302090: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-02-06 18:13:34.333438: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8100
2022-02-06 18:13:34.341992: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-02-06 18:13:35.209350: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


 1/60 [..............................] - ETA: 29:07 - loss: 0.8482 - bpp: 0.0996 - mse: 0.0015

2022-02-06 18:13:40.032100: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-02-06 18:13:40.032347: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.


 2/60 [>.............................] - ETA: 2:09 - loss: 0.5499 - bpp: 0.0918 - mse: 8.9483e-04

2022-02-06 18:13:41.424824: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-02-06 18:13:41.438158: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1743] CUPTI activity buffer flushed
2022-02-06 18:13:41.559853: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 3518 callback api events and 3482 activity events. 
2022-02-06 18:13:41.673007: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-02-06 18:13:41.837747: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/fit/20220206-181258/train/plugins/profile/2022_02_06_18_13_41
2022-02-06 18:13:41.916400: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/fit/20220206-181258/train/plugins/profile/2022_02_06_18_13_41/bb6aa9fd969d.trace.json.gz
2022-02-06 18:13:42.095813: I tensorflow/core/profiler/rpc/client/save_profile.cc

60/60 [==============================] - 39s 161ms/step - loss: 1.1020 - bpp: 0.0995 - mse: 0.0020
[MemoryCallback]:  6416316

Epoch 00001: loss improved from inf to 1.10205, saving model to checkpoints_Lagrangian_256_10/
Epoch 2/10000
60/60 [==============================] - 7s 116ms/step - loss: 0.9410 - bpp: 0.0974 - mse: 0.0016
[MemoryCallback]:  6425328

Epoch 00002: loss improved from 1.10205 to 0.94102, saving model to checkpoints_Lagrangian_256_10/
Epoch 3/10000
60/60 [==============================] - 7s 115ms/step - loss: 0.9437 - bpp: 0.0957 - mse: 0.0017
[MemoryCallback]:  6514196

Epoch 00003: loss did not improve from 0.94102
Epoch 4/10000
60/60 [==============================] - 7s 112ms/step - loss: 1.0329 - bpp: 0.0982 - mse: 0.0018
[MemoryCallback]:  6552036

Epoch 00004: loss did not improve from 0.94102
Epoch 5/10000
60/60 [==============================] - 7s 112ms/step - loss: 0.9816 - bpp: 0.0958 - mse: 0.0017
[MemoryCallback]:  6553420

Epoch 00005: loss did not

In [11]:
path = load.load_random_path("/mnt/WindowsDev/Developer/tensorflow-wavelets/folder_cloud.npy")
i=0
out_bin = "/workspaces/tensorflow-wavelets/Development/OpenDVC/Test_com/test{}.bin".format(i)
out_decom = "/workspaces/tensorflow-wavelets/Development/OpenDVC/Test_com/testdcom{}.png".format(i)
p_on_test = "/workspaces/tensorflow-wavelets/Development/OpenDVC/Test_com/test_p_frame{}.png".format(i)
i_on_test = "/workspaces/tensorflow-wavelets/Development/OpenDVC/Test_com/test_i_frame{}.png".format(i)

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)

OpenDVC.write_png(p_on_test, OpenDVC.read_png_crop(p_frame, 240, 240))
OpenDVC.write_png(i_on_test, OpenDVC.read_png_crop(i_frame, 240, 240))

OpenDVC.compress(model, i_frame, p_frame, out_bin, 240, 240)
OpenDVC.decompress(model, i_frame, out_bin, out_decom, 240, 240)

/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00070/0362/im1.png
compress
in the compress
Instructions for updating:
Use fn_output_signature instead
in decompress


In [12]:
model.save(save_name, save_format="tf")

in the compress
in decompress


INFO:tensorflow:Assets written to: model_save_l_256_10/assets


INFO:tensorflow:Assets written to: model_save_l_256_10/assets
